# Create Azure SQL Database

## Description

Run this notebook to create a Azure SQL Database in a new Azure SQL Server in the cloud and then configure a server-level firewall rule it. For more information, see [Use PowerShell to create a single database and configure a server-level firewall rule](https://docs.microsoft.com/en-us/azure/azure-sql/database/scripts/create-and-configure-database-powershell).

| Line | Variable | Description | Example |
| --- | --- | --- | --- |
| 1 | Subscription | Specify the name or ID of the Azure subscription to create Azure resources in | "ContosoCorp\_Infra" |
| 2 | ResourceGroup | Choose a name to logically group the Azure resources | "ContosoBackend" |
| 3 | Location | Name of geographic location (See the [Appendices](.\................\Program%20Files\Azure%20Data%20Studio\resources\app\out\vs\code\electron-browser\Appendices.ipynb) for more information) | "EastUS2" |
| 4 | ServerName | The logical server name has to be unique in the system | "contoso-srv1" |
| 5 | AdminLogin | SQL admin login to create | "SqlAdmin" |
| 6 | Password | Temporary password to use (change in the portal for greater security) | "Temp123" |
| 7 | StartIp | Starting IP to begin the IP firewall rule | "0.0.0.0" for everyone |
| 8 | EndIp | Ending IP to end the IP firewall rule | "0.0.0.0" for everyone |
| 9 | DatabaseName | The sample database name | "ContosoDb1" |
| 10 | DbEdition | Database edition. Allowed values include: _Basic_, _Standard_, _Premium_, _GeneralPurpose_, _BusinessCritical_, _Hyperscale_ | GeneralPurpose |
| 11 | DbCores | Integer for number of vcores to utilize | 2 |
| 12 | DbComputeGen | The compute generation component for vcores. Allowed values include: _Gen4_, _Gen5_. | Gen5 |
| 13 | DbMinCapacity | Minimum capacity of vcores to utilize, integer only | 2 |

In [ ]:
$Subscription       =   ""
$ResourceGroup      =   ""
$Location           =   ""
$ServerName         =   ""
$AdminLogin         =   ""
$Password           =   ""
$StartIp            =   ""
$EndIp              =   ""
$DatabaseName       =   ""
$DbEdition          =   ""
$DbCores            =   ""
$DbComputeGen       =   ""
$DbMinCapacity=     =   ""

## Notebook Steps

Steps of this procedure include:
1. Connect to Azure subscription
1. Provision resource group for SQL Managed Instance
2. Create Sql Server
3. Provision firewall rules to allow access
4. Create Sql Database

### Connect to Azure
Prompt for Azure account credentials in a dialog window outside of ADS.

In [ ]:
Connect-AzAccount

### Get Subscription
If a subscription is not specified, open a dialog with list of subscriptions. Selecting one will set that subscription for rest of the notebook.

In [ ]:
if (!$Subscription)
{
    $Subscription = Get-AzSubscription | Out-GridView -PassThru
}

Set-AzContext -SubscriptionName $Subscription

### Resource Group Provisioning

If the specified Azure Resource Group cannot be found, _New-AzResourceGroup_ will create new resource group in the given subscription.

In [ ]:
$rg = Get-AzResourceGroup | Where ResourceGroupName -eq $ResourceGroup

if (!$rg)
{
    # Need to create a new resource group
    Write-Output "Resource Group $ResourceGroup does not exist. Creating..."
    $rg = New-AzResourceGroup -Name $ResourceGroup -Location $Location
}

Write-Output "Using Resource Group:"
$rg | Format-Table

### Create SQL Server

_New-AzSqlServer_ will create an Azure Sql Server instance to logically group databases in.

In [ ]:
Write-Output "Creating SqlServer with name $ServerName"
New-AzSqlServer -ResourceGroupName $ResourceGroup `
   -ServerName $ServerName `
   -Location $Location `
   -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
   -ArgumentList $AdminLogin, $(ConvertTo-SecureString -String $Password -AsPlainText -Force))

### Create a Server Firewall Rule

_New-AzSqlServerFirewallRule_ creates a new firewall rule for the new SQL Server that allows a range of specified IPs.

In [ ]:
Write-Output "Configuring firewall for Sql Server"
New-AzSqlServerFirewallRule -ResourceGroupName $ResourceGroup `
   -ServerName $ServerName `
   -FirewallRuleName "AllowedIPs" -StartIpAddress $StartIp -EndIpAddress $EndIp

### Create SQL Database
_New-AzSqlDatabase_ command will create a new database in the server.

In [ ]:
New-AzSqlDatabase -ResourceGroupName $ResourceGroup `
   -ServerName $ServerName `
   -DatabaseName $DatabaseName `
   -Edition $DbEdition `
   -VCore $DbCores `
   -ComputeGeneration $DbComputeGen `
   -MinimumCapacity $DbMinCapacity

### Get Database Details

_Get-AzSqlDatabase_ command gets database details on the server specified.

In [ ]:
Get-AzSqlDatabase -ResourceGroupName $ResourceGroup -ServerName $ServerName -DatabaseName $DatabaseName

## Next Steps

### Update SQL Server Password
Do not forget to open the <a href="https://portal.azure.com">Azure portal</a> to the specified Resource Group in an external browser and change the new server's password. It is not recommended to keep the database password used in this notebook because it stores results as plain text. 

### More Automation
Use ADS Notebooks to write custom scripts for:
* Commonly used Extract, Transform, Load operations
* Data migration and validation
* Storage backup
* Data integration

### Related Notebooks
For existing database owners, Migrate a database to Azure SQLDB <a href="..\offline-migration\db-to-SQLDB.ipynb">using this notebook</a>. 